In [ ]:
library(Seurat)
library(ggplot2)
library(dplyr)
library(patchwork)
library(stringr)
library(ggplot2)

In [ ]:
# https://drive.google.com/file/d/1XbDy0A_rLt-kTDQvtqyCCGJ__qxdym7e/view?usp=drive_link
pseudo_seu <- readRDS(file = "./pseudo_Seu_obj_2.rds")

## perform DEG analysis on sex

In [121]:
pseudo_seu$sex <- stringr::str_split(pseudo_seu$sample_name,"-",simplify = T)[,2]
pseudo_seu$celltype.sex <- paste(pseudo_seu$celltype,pseudo_seu$sex,sep = "_")
pseudo_seu$group.celltype.sex <- paste(str_split(pseudo_seu$sample_name,"-",simplify = T)[,1],
                                 pseudo_seu$celltype.sex,sep = "_")


In [ ]:
Idents(pseudo_seu) <- "group.celltype.sex"
pseudo_de_sex <- list()
for(i in unique(pseudo_seu$celltype)){
    for(j in c("iso","soc")){
        bulk.de <- FindMarkers(object = pseudo_seu,
                              ident.1 = paste0(j,"_",i,"_F"),
                              ident.2 = paste0(j,"_",i,"_M"),
                               test.use = "DESeq2")
    
        pseudo_de_sex[[paste0(i,"_",j)]] <- bulk.de
    }
    
}

In [68]:
filename = str_replace_all(string =names(pseudo_de_sex) ,pattern = "\\/" ,replacement = "")

In [ ]:
## save all gene list
lapply(seq_len(length(names(pseudo_de_sex))),function(x){
            pseudo_de_sex[[x]] %>%
            write.table(,file=paste0("./FvsM_FC0.3.DEG.list_2/",filename[x],"_F_vs_M_total_gene.xls"),sep="\t",quote=F)
    }  
)

In [ ]:
lapply(seq_len(length(names(pseudo_de_sex))), function(x) {
    if (sum(pseudo_de_sex[[x]]$p_val_adj < 0.05 & abs(pseudo_de_sex[[x]]$avg_log2FC) > 0.3,na.rm = TRUE) > 0) {
        filtered_data <- pseudo_de_sex[[x]] %>%
            filter(p_val_adj < 0.05, abs(avg_log2FC) > 0.3)
        
        write.table(filtered_data, 
                    file = paste0("./FvsM_FC0.3.DEG.list_2/", filename[x], "_F_vs_M_DEG.xls"), 
                    sep = "\t", 
                    quote = FALSE, 
                    row.names = TRUE)
    }
})

In [125]:
saveRDS(pseudo_de_sex,file = "./pseudo_sex_DESeq2_DEG_2.rds")

In [1]:
pseudo_de_sex <- readRDS(file = "./pseudo_sex_DESeq2_DEG_2.rds")

## GO analysis for sex group DEGS

In [ ]:
DEG_ls <- lapply(pseudo_de_sex, function(x){ x %>% filter(p_val_adj<0.05,abs(avg_log2FC) >0.3) %>% row.names()})
names(DEG_ls) = str_replace_all(string =names(DEG_ls) ,pattern = "\\/" ,replacement = "")
GO_ls <- list()
for(i in seq_len(length(DEG_ls))){
    if(length(DEG_ls[[i]])>5){
           go_res <- enrichGO(DEG_ls[[i]],OrgDb=org.Rn.eg.db,keyType="SYMBOL",ont="BP",pvalueCutoff  = 0.05, qvalueCutoff  = 1)
          if(!is.null(go_res@result)){
              GO_ls[names(DEG_ls)[i]] <- go_res
          }
           }
}
cp_GO_ls <- GO_ls

write.xlsx(lapply(cp_GO_ls,function(df){df@result}),file="./FvsM_DEG_GO_2/FvsM_DEG_FC0.3_GO.xlsx",asTable=T)


In [ ]:
pdf("/project/zhuzhuzhang/lyang/scaleBio_RNA_rat_brain/out_mRatBN/FvsM_DEG_GO_2/FvsM_DEG_FC0.3_GO_dotplot.pdf",onefile=TRUE)
lapply(seq_along(GO_ls),
       function(i){
           if(!is.null(GO_ls[[i]])){
           if(dim(GO_ls[[i]]@result)[1]>0){
               if(sum(GO_ls[[i]]@result$p.adjust<0.05)>0){
                   GO_ls[[i]]@result %>%
                   filter(p.adjust<0.05) %>%
                       mutate(ratio=as.numeric(str_split(GeneRatio,"/",simplify=T)[,1])/as.numeric(str_split(GeneRatio,"/",simplify=T)[,2])) %>%
                       slice_head(n=15) %>%
                        arrange(ratio) %>%
                        mutate(Description=factor(Description,levels=Description)) %>%
                       ggplot(aes(x=ratio,y=Description))+
                       geom_point(aes(color=p.adjust,size=ratio))+
                       labs(title=paste0(names(GO_ls)[i]," ","F vs M ","(",length(DEG_ls[[i]]) ,")"))+
                       scale_y_discrete(labels = function(x) str_wrap(x, width = 50))+
                        scale_color_viridis(option = "D")
                   }
               
           }
            }
           }
)
dev.off()